In [2]:
import pandas as pd

In [3]:
df=pd.read_csv("faa_ai_prelim.csv")

In [4]:
print(df.shape)
print(df.head())
print(df.info())

(83, 42)
  UPDATED ENTRY_DATE EVENT_LCL_DATE EVENT_LCL_TIME LOC_CITY_NAME  \
0      No  19-FEB-16      19-FEB-16      00:45:00Z    MARSHVILLE   
1      No  19-FEB-16      18-FEB-16      23:55:00Z     TAVERNIER   
2      No  19-FEB-16      18-FEB-16      22:14:00Z       TRENTON   
3      No  19-FEB-16      18-FEB-16      17:10:00Z     ASHEVILLE   
4      No  19-FEB-16      18-FEB-16      00:26:00Z     TALKEETNA   

   LOC_STATE_NAME LOC_CNTRY_NAME  \
0  North Carolina            NaN   
1         Florida            NaN   
2      New Jersey            NaN   
3  North Carolina            NaN   
4          Alaska            NaN   

                                            RMK_TEXT EVENT_TYPE_DESC  \
0  AIRCRAFT CRASHED INTO TREES, THE 1 PERSON ON B...        Accident   
1  AIRCRAFT ON LANDING WENT OFF THE END OF THE RU...        Incident   
2  AIRCRAFT ON FINAL SUSTAINED A BIRD STRIKE, LAN...        Incident   
3  AIRCRAFT ON LANDING, GEAR COLLAPSED, ASHEVILLE...        Incident   
4  AI

In [5]:
# Convert date and time columns to datetime
df['EVENT_DATETIME'] = pd.to_datetime(df['EVENT_LCL_DATE'] + ' ' + df['EVENT_LCL_TIME'], errors='coerce')


C:\Users\Dushyant Yadav\AppData\Local\Temp\ipykernel_86128\3094786609.py:2: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df['EVENT_DATETIME'] = pd.to_datetime(df['EVENT_LCL_DATE'] + ' ' + df['EVENT_LCL_TIME'], errors='coerce')


In [6]:
df.head()

,UPDATED,ENTRY_DATE,EVENT_LCL_DATE,EVENT_LCL_TIME,LOC_CITY_NAME,LOC_STATE_NAME,LOC_CNTRY_NAME,RMK_TEXT,EVENT_TYPE_DESC,FSDO_DESC,...,PAX_INJ_MINOR,PAX_INJ_SERIOUS,PAX_INJ_FATAL,PAX_INJ_UNK,GRND_INJ_NONE,GRND_INJ_MINOR,GRND_INJ_SERIOUS,GRND_INJ_FATAL,GRND_INJ_UNK,EVENT_DATETIME
0,No,19-FEB-16,19-FEB-16,00:45:00Z,MARSHVILLE,North Carolina,NaN,"AIRCRAFT CRASHED INTO TREES, THE 1 PERSON ON B...",Accident,FAA Charlotte FSDO-68,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2016-02-19 00:45:00+00:00
1,No,19-FEB-16,18-FEB-16,23:55:00Z,TAVERNIER,Florida,NaN,AIRCRAFT ON LANDING WENT OFF THE END OF THE RU...,Incident,FAA Miami FSDO-19,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2016-02-18 23:55:00+00:00
2,No,19-FEB-16,18-FEB-16,22:14:00Z,TRENTON,New Jersey,NaN,"AIRCRAFT ON FINAL SUSTAINED A BIRD STRIKE, LAN...",Incident,FAA Philadelphia FSDO-17,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2016-02-18 22:14:00+00:00
3,No,19-FEB-16,18-FEB-16,17:10:00Z,ASHEVILLE,North Carolina,NaN,"AIRCRAFT ON LANDING, GEAR COLLAPSED, ASHEVILLE...",Incident,FAA Charlotte FSDO-68,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2016-02-18 17:10:00+00:00
4,No,19-FEB-16,18-FEB-16,00:26:00Z,TALKEETNA,Alaska,NaN,"AIRCRAFT ON LANDING, NOSE GEAR COLLAPSED, TALK...",Incident,FAA Anchorage FSDO-03,...,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2016-02-18 00:26:00+00:00


In [7]:
cols_needed = [
    'ACFT_MAKE_NAME',   # Aircraft make name
    'LOC_STATE_NAME',   # State name
    'ACFT_MODEL_NAME',  # Aircraft model name
    'RMK_TEXT',         # Text information
    'FLT_PHASE',        # Flight phase
    'EVENT_TYPE_DESC',  # Event description type
    'FATAL_FLAG',       # Fatal flag
    'ACFT_DMG_DESC'     # Aircraft damage description
]
df = df[cols_needed + ['EVENT_DATETIME']]  # Keep new datetime too


In [8]:
# Fill missing Fatal Flag with 'No'
df['FATAL_FLAG'] = df['FATAL_FLAG'].fillna('No')

# Fill FLT_PHASE and ACFT_DMG_DESC with mode
for col in ['FLT_PHASE', 'ACFT_DMG_DESC']:
    df[col] = df[col].fillna(df[col].mode()[0])


In [9]:
print(df.isna().sum())


ACFT_MAKE_NAME     5
LOC_STATE_NAME     0
ACFT_MODEL_NAME    4
RMK_TEXT           0
FLT_PHASE          0
EVENT_TYPE_DESC    0
FATAL_FLAG         0
ACFT_DMG_DESC      0
EVENT_DATETIME     0
dtype: int64


In [10]:
# Remove rows with missing aircraft names
df = df[df['ACFT_MAKE_NAME'].notna()]

# Drop columns with more than 75% missing values
threshold = len(df) * 0.25
df = df.dropna(thresh=threshold, axis=1)


In [11]:
print("After cleaning:", df.shape)


After cleaning: (78, 9)


In [12]:
print(df.isna().sum())

ACFT_MAKE_NAME     0
LOC_STATE_NAME     0
ACFT_MODEL_NAME    1
RMK_TEXT           0
FLT_PHASE          0
EVENT_TYPE_DESC    0
FATAL_FLAG         0
ACFT_DMG_DESC      0
EVENT_DATETIME     0
dtype: int64


In [13]:
fatal_cases = df[df['FATAL_FLAG'] == 'Yes']
print(fatal_cases)


    ACFT_MAKE_NAME  LOC_STATE_NAME ACFT_MODEL_NAME  \
0            BEECH  North Carolina              36   
53           PIPER         Florida            PA28   
55   FLIGHT DESIGN      California            CTLS   
79  NORTH AMERICAN         Arizona             F51   
80        CHAMPION      California           8KCAB   
81           BEECH      California              35   
82          CESSNA         Alabama             182   

                                             RMK_TEXT      FLT_PHASE  \
0   AIRCRAFT CRASHED INTO TREES, THE 1 PERSON ON B...  UNKNOWN (UNK)   
53  AIRCRAFT CRASHED UNDER UNKNOWN CIRCUMSTANCES. ...  UNKNOWN (UNK)   
55  AIRCRAFT CRASHED UNDER UNKNOWN CIRCUMSTANCES A...  UNKNOWN (UNK)   
79  AIRCRAFT CRASHED UNDER UNKNOWN CIRCUMSTANCES, ...  UNKNOWN (UNK)   
80  N9872R, BEECH M35 AIRCRAFT, AND N5057G, BELLAN...  UNKNOWN (UNK)   
81  N9872R, BEECH M35 AIRCRAFT, AND N5057G, BELLAN...  UNKNOWN (UNK)   
82  N784CP AIRCRAFT CRASHED INTO A WOODED AREA NEA...  UNKNOWN 

In [14]:
df_encoded = pd.get_dummies(df, columns=['ACFT_DMG_DESC'], drop_first=True)


In [15]:
# Define keywords and corresponding phase
phase_keywords = {
    'LANDING': 'LANDING',
    'TAKEOFF': 'TAKEOFF',
    'CRUISE': 'CRUISE',
    'APPROACH': 'APPROACH'
}

def infer_phase(text):
    if pd.isna(text):
        return 'UNKNOWN'
    text = text.upper()
    for keyword, phase in phase_keywords.items():
        if keyword in text:
            return phase
    return 'UNKNOWN'

df['FLIGHT_PHASE_TEXT'] = df['RMK_TEXT'].apply(infer_phase)

# Compare with existing FLT_PHASE
comparison = df[['FLT_PHASE', 'FLIGHT_PHASE_TEXT']]
print(comparison.head())


        FLT_PHASE FLIGHT_PHASE_TEXT
0   UNKNOWN (UNK)           UNKNOWN
1   LANDING (LDG)           LANDING
2  APPROACH (APR)           UNKNOWN
3   LANDING (LDG)           LANDING
4   LANDING (LDG)           LANDING


In [16]:
df.to_csv("aviation_data_processed.csv", index=False)
